In [23]:
import pandas as pd
from data_mountain_query.query import get_ambient_tweets
from data_mountain_query.connection import get_connection
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from datetime import timedelta
import plotly.express as px


In [24]:
games = pd.read_csv("/Users/elisabethkollrack/Thesis/EK-thesis/games.csv")
games = games[games['game_type'] == 'REG']
games['gameday'] = pd.to_datetime(games['gameday'], format='%m/%d/%y')

# NE 2013 games
ne_games = games[
    (games['season'] >= 2013) & (games['season'] <= 2017) &
    ((games['home_team'] == 'NE') | (games['away_team'] == 'NE'))
].sort_values(['season', 'gameday'])


In [25]:
collection, client = get_connection(geotweets=True)

Connecting on mgmt1.vacc.uvm.edu


In [26]:
%%capture

all_tweets = []
for index, game in ne_games.iterrows():
    gameday = game['gameday']
    
    # Determine opponent
    if game['home_team'] == 'NE':
        opponent = game['away_team']
    else:
        opponent = game['home_team']

    anchors = [
        "#newenglandpatriots", 
        "#patriots",
        f"#NEvs{opponent}",
        f"#{opponent}vsNE"
    ]

    start_date = gameday - timedelta(days=3)
    end_date = gameday + timedelta(days=3)
    dates = pd.date_range(start_date, end_date, freq='D')

    for anchor in anchors:
        tweets_list1 = [t for t in get_ambient_tweets(anchor, dates, collection)]
        all_tweets.extend(tweets_list1)



In [27]:
geo_df = pd.DataFrame(all_tweets)

# Extract lon and lat from the 'geo' column
geo_df['lon'] = geo_df['geo'].apply(lambda x: x['coordinates'][0] if isinstance(x, dict) else None)
geo_df['lat'] = geo_df['geo'].apply(lambda x: x['coordinates'][1] if isinstance(x, dict) else None)

geo_df.head()

,_id,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,...,twitter_entities,twitter_filter_level,twitter_lang,retweetCount,gnip,twitter_extended_entities,inReplyTo,long_object,lon,lat
0,5e447d12f765e1127af9bbb3,Thu Sep 05 15:22:32 +0000 2013,375640096269017100,375640096269017089,use my patriots jersey on game days \n#SoIKnow...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-97.451755,25.900049
1,5e448d4df765e1127a1c1ca3,Sun Sep 08 14:30:46 +0000 2013,376714232500588540,376714232500588544,#nflgameday apparel #patriots #newenglandpatri...,"<a href=""http://instagram.com"" rel=""nofollow"">...",False,NaN,None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-71.269603,41.675478
2,5e4488dbf765e1127a129951,Sun Sep 08 17:05:37 +0000 2013,376753202013560800,376753202013560833,Lets go Patriots!!!!!!!! #GameDay #Patriots @P...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-117.827185,33.698094
3,5e448d5ef765e1127a1c5c83,Sun Sep 08 14:56:08 +0000 2013,376720616390291460,376720616390291456,Lets Go Patriots!!#PatsNation #Patriots #brady,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-78.879703,43.047750
4,5e448cedf765e1127a1b4b7c,Sun Sep 08 13:33:23 +0000 2013,376699791750594560,376699791750594561,Then later patriots vs bills. #Patriots #letsgo,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-75.265707,43.024112


In [28]:
cities = gpd.read_file("/Users/elisabethkollrack/Thesis/EK-Thesis/tl_2024_us_cbsa/tl_2024_us_cbsa.shp")

tweets_gdf = gpd.GeoDataFrame(
    geo_df, 
    geometry=gpd.points_from_xy(geo_df.lon, geo_df.lat),
    crs="EPSG:4326"
)

# Spatial join of tweets inside metros
tweets_with_city = gpd.sjoin(
    tweets_gdf,
    cities,
    how='left',
    predicate='within'
)

# Keep only major metropolitan statistical areas (M1)
tweets_with_city = tweets_with_city[tweets_with_city["LSAD"] == "M1"]

tweets_with_city['NAME'].value_counts()


/var/folders/cg/kg987wfn3_977vnx3pkwqg0m0000gn/T/ipykernel_17764/1156845772.py:10: UserWarning:

CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: EPSG:4269




NAME
Boston-Cambridge-Newton, MA-NH                  1111
New York-Newark-Jersey City, NY-NJ               350
Los Angeles-Long Beach-Anaheim, CA               270
Providence-Warwick, RI-MA                        254
Washington-Arlington-Alexandria, DC-VA-MD-WV     207
                                                ... 
Slidell-Mandeville-Covington, LA                   1
Joplin, MO-KS                                      1
Minot, ND                                          1
Valdosta, GA                                       1
Chambersburg, PA                                   1
Name: count, Length: 314, dtype: int64

In [31]:
cities.columns


Index(['CSAFP', 'CBSAFP', 'GEOID', 'GEOIDFQ', 'NAME', 'NAMELSAD', 'LSAD',
       'MEMI', 'MTFCC', 'ALAND', 'AWATER', 'INTPTLAT', 'INTPTLON', 'geometry'],
      dtype='object')

In [29]:


# Summarize tweet counts by metro area
metro_summary = tweets_with_city.groupby('NAME').agg({
    'id': 'count',
    'lat': 'mean',
    'lon': 'mean'
}).reset_index()
metro_summary = metro_summary.rename(columns={'id': 'count'})

# Filter for metros with minimum tweets
metro_summary = metro_summary[metro_summary['count'] >= 10]  # Adjust threshold


# Create the map
fig = px.scatter_geo(metro_summary,
                     lat="lat",
                     lon="lon", 
                     size="count",
                     hover_name="NAME",
                     hover_data={"count": True, "lat": False, "lon": False},
                     title="Patriots Tweet Activity by Metropolitan Area (2013-2017 non-normalized populations)",
                     scope="usa")

fig.update_geos(
    scope="usa",
    showland=True,
    landcolor="lightgray",
    showocean=True,
    oceancolor="azure",
    showlakes=True,
    lakecolor="azure",
    showrivers=True,
    rivercolor="azure"
)

fig.update_layout(
    title_x=0.5,
    title_font_size=20,
    geo=dict(
        bgcolor='rgba(0,0,0,0)',
        lakecolor='azure',
        landcolor='lightgray'
    )
)

fig.show()

In [30]:
# save as html
fig.write_html("patriots_tweet_activity_metro_2013_2017_non_normalized.html")
